In [1]:
import pandas as pd
import numpy as np
drug = pd.read_csv("../data/eICU/infusiondrug.csv")
diagnosis = pd.read_csv("../data/eICU/diagnosis.csv")
patients_weight = pd.read_csv("../data/eICU/patient.csv")[["patientunitstayid", "admissionweight"]]
drug = drug.merge(patients_weight, on="patientunitstayid")

C:\Users\PC\AppData\Local\Temp\ipykernel_956\1778484243.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  drug = pd.read_csv("../data/eICU/infusiondrug.csv")


In [2]:
patient = pd.read_csv("../data/eICU/patient.csv")
patient

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,...,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851
1,141178,128927,Female,52,Caucasian,60,83,NaN,162.6,08:56:00,...,Emergency Department,1,admit,54.4,54.4,09:18:00,8,Step-Down Unit (SDU),Alive,002-33870
2,141179,128927,Female,52,Caucasian,60,83,NaN,162.6,08:56:00,...,ICU to SDU,2,stepdown/other,NaN,60.4,19:20:00,2042,Home,Alive,002-33870
3,141194,128941,Male,68,Caucasian,73,92,"Sepsis, renal/UTI (including bladder)",180.3,18:18:40,...,Floor,1,admit,73.9,76.7,15:31:00,4813,Floor,Alive,002-5276
4,141196,128943,Male,71,Caucasian,67,109,NaN,162.6,20:21:00,...,ICU to SDU,2,stepdown/other,NaN,63.2,22:23:00,1463,Floor,Alive,002-37665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200854,3353235,2743084,Male,50,Caucasian,458,1109,"CHF, congestive heart failure",175.3,04:55:00,...,Emergency Department,1,admit,90.0,99.2,23:18:00,1069,Telemetry,Alive,035-16382
200855,3353237,2743086,Female,79,Caucasian,458,1106,"Embolus, pulmonary",162.6,01:45:00,...,Direct Admit,1,admit,78.4,81.4,23:08:00,1269,Step-Down Unit (SDU),Alive,035-751
200856,3353251,2743099,Male,73,African American,458,1104,Cardiac arrest (with or without respiratory ar...,177.8,12:51:00,...,Emergency Department,1,admit,102.0,96.2,23:16:00,16259,Telemetry,Alive,035-5166
200857,3353254,2743102,Male,81,Caucasian,459,1108,"Bleeding, lower GI",185.4,07:43:00,...,Emergency Department,1,admit,83.9,92.9,19:25:00,431,Step-Down Unit (SDU),Alive,035-19511


In [3]:
# add hospitalID and unitType and wardID to the drug table corelated to patientunitstayid
drug["hospitalid"] = drug["patientunitstayid"].map(patient.set_index("patientunitstayid")["hospitalid"])
drug["unittype"] = drug["patientunitstayid"].map(patient.set_index("patientunitstayid")["unittype"])
drug["wardid"] = drug["patientunitstayid"].map(patient.set_index("patientunitstayid")["wardid"])



In [4]:
# filter drug from NA in drugname, patientunitstayid, hospitalid, unittype, wardid
drug = drug[~drug['drugname'].isna()]
drug = drug[~drug['patientunitstayid'].isna()]
drug = drug[~drug['hospitalid'].isna()]
drug = drug[~drug['unittype'].isna()]
drug = drug[~drug['wardid'].isna()]

#filter drug to include only drugnames that start with "Norepinephrine"
drug = drug[drug['drugname'].str.startswith("Norepinephrine")]

In [27]:
# sort drug by patientunitstayid and drugstartoffset
drug = drug.sort_values(by=["patientunitstayid", "infusionoffset"])
drug

,infusiondrugid,patientunitstayid,infusionoffset,drugname,drugrate,infusionrate,drugamount,volumeoffluid,patientweight,admissionweight,hospitalid,unittype,wardid
4,1969910,242082,55,Norepinephrine (mcg/min),2.13,8.0,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
2,1968206,242082,125,Norepinephrine (mcg/min),7.00,26.3,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
10,1985898,242082,230,Norepinephrine (mcg/min),9.07,34.0,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
12,1972141,242082,275,Norepinephrine (mcg/min),10.00,37.5,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
1,1998443,242082,425,Norepinephrine (mcg/min),10.93,41.0,4.0,250.0,NaN,90.7,108,Med-Surg ICU,136
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4803535,75122092,3353251,3823,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,15.00,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104
4803681,75308975,3353251,3968,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,16.00,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104
4803654,75246300,3353251,4043,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,12.00,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104
4803592,75148203,3353251,4358,Norepinephrine MAX 32 mg Dextrose 5% 250 ml (m...,10.00,NaN,NaN,NaN,NaN,102.0,458,Cardiac ICU,1104


In [6]:
# plot histogram of number of differnt patientsunitstayid for every hospitalid for each unittype and total
drug.groupby(["hospitalid", "unittype"])["patientunitstayid"].nunique().sort_values()
# drug.groupby("hospitalid")["patientunitstayid"].nunique().sort_values()

hospitalid  unittype    
342         Med-Surg ICU      1
151         Med-Surg ICU      1
265         MICU              1
156         Med-Surg ICU      1
438         Med-Surg ICU      1
                           ... 
157         Med-Surg ICU    375
167         Med-Surg ICU    384
188         MICU            430
300         Med-Surg ICU    452
199         Med-Surg ICU    525
Name: patientunitstayid, Length: 195, dtype: int64

In [7]:
# for each hospitalid, give me the number of patientsunitstayid for each unittype and their sum
hospis = drug.groupby(["hospitalid", "unittype"])["patientunitstayid"].nunique().sort_values().unstack().fillna(0).assign(sum=lambda x: x.sum(axis=1)).sort_values(by="sum", ascending=False)


In [8]:
hospis[hospis['sum'] > 250].size


207

In [9]:
bp = pd.read_csv("../preprocess/filtered_bp_eicu.csv")

In [10]:
# sort bp by stay_id and cur_bp_time
bp = bp.sort_values(by=["stay_id", "cur_bp_time"])
bp

,Unnamed: 0,stay_id,cur_bp_time,cur_bp
92,161,141227,597,92.0
105,191,141227,602,92.0
136,263,141227,607,88.0
120,227,141227,612,86.0
141,275,141227,617,95.0
...,...,...,...,...
3328428,950941,3353251,12179,65.0
3328499,951041,3353251,12184,65.0
3328497,951037,3353251,12189,64.0
3328502,951045,3353251,12194,60.0


In [11]:
# filter bp for patients that have norepinephrine
bp = bp[bp["stay_id"].isin(drug["patientunitstayid"])]
bp

,Unnamed: 0,stay_id,cur_bp_time,cur_bp
435480,1494922,243334,51,199.0
435501,1494950,243334,56,199.0
435504,1494954,243334,61,199.0
435470,1494906,243334,66,199.0
435474,1494910,243334,71,199.0
...,...,...,...,...
3328428,950941,3353251,12179,65.0
3328499,951041,3353251,12184,65.0
3328497,951037,3353251,12189,64.0
3328502,951045,3353251,12194,60.0


In [12]:
# add column of interval between current bp time and next bp time for each patient
bp["next_bp_time"] = bp.groupby("stay_id")["cur_bp_time"].shift(-1)
bp["interval"] = bp["next_bp_time"] - bp["cur_bp_time"]
bp

C:\Users\PC\AppData\Local\Temp\ipykernel_956\4226854582.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bp["next_bp_time"] = bp.groupby("stay_id")["cur_bp_time"].shift(-1)
C:\Users\PC\AppData\Local\Temp\ipykernel_956\4226854582.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bp["interval"] = bp["next_bp_time"] - bp["cur_bp_time"]


,Unnamed: 0,stay_id,cur_bp_time,cur_bp,next_bp_time,interval
435480,1494922,243334,51,199.0,56.0,5.0
435501,1494950,243334,56,199.0,61.0,5.0
435504,1494954,243334,61,199.0,66.0,5.0
435470,1494906,243334,66,199.0,71.0,5.0
435474,1494910,243334,71,199.0,76.0,5.0
...,...,...,...,...,...,...
3328428,950941,3353251,12179,65.0,12184.0,5.0
3328499,951041,3353251,12184,65.0,12189.0,5.0
3328497,951037,3353251,12189,64.0,12194.0,5.0
3328502,951045,3353251,12194,60.0,12199.0,5.0


In [13]:
# add amount of NOR
drug["drugrate"] = pd.to_numeric(drug["drugrate"], errors='coerce')
avg_nor = drug.groupby("patientunitstayid")["drugamount", "drugrate"].mean()
avg_nor = avg_nor.reset_index()
avg_nor
bp = pd.merge(bp, avg_nor, how="left", left_on="stay_id", right_on="patientunitstayid")
bp.drop('patientunitstayid', axis=1, inplace=True)
bp


C:\Users\PC\AppData\Local\Temp\ipykernel_956\28898674.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  avg_nor = drug.groupby("patientunitstayid")["drugamount", "drugrate"].mean()


,Unnamed: 0,stay_id,cur_bp_time,cur_bp,next_bp_time,interval,drugamount,drugrate
0,1494922,243334,51,199.0,56.0,5.0,4.0,4.375000
1,1494950,243334,56,199.0,61.0,5.0,4.0,4.375000
2,1494954,243334,61,199.0,66.0,5.0,4.0,4.375000
3,1494906,243334,66,199.0,71.0,5.0,4.0,4.375000
4,1494910,243334,71,199.0,76.0,5.0,4.0,4.375000
...,...,...,...,...,...,...,...,...
3941958,950941,3353251,12179,65.0,12184.0,5.0,NaN,17.939394
3941959,951041,3353251,12184,65.0,12189.0,5.0,NaN,17.939394
3941960,951037,3353251,12189,64.0,12194.0,5.0,NaN,17.939394
3941961,951045,3353251,12194,60.0,12199.0,5.0,NaN,17.939394


In [14]:
# add age
bp["age"] = pd.to_numeric(bp["stay_id"].map(patient.set_index("patientunitstayid")["age"]), errors='coerce')

In [15]:
# add column of hospitalid, unittype and wardid to the bp table corelated to patientunitstayid
bp["hospitalid"] = bp["stay_id"].map(patient.set_index("patientunitstayid")["hospitalid"])
bp["unittype"] = bp["stay_id"].map(patient.set_index("patientunitstayid")["unittype"])
bp["wardid"] = bp["stay_id"].map(patient.set_index("patientunitstayid")["wardid"])
bp

,Unnamed: 0,stay_id,cur_bp_time,cur_bp,next_bp_time,interval,drugamount,drugrate,age,hospitalid,unittype,wardid
0,1494922,243334,51,199.0,56.0,5.0,4.0,4.375000,89.0,79,Med-Surg ICU,133
1,1494950,243334,56,199.0,61.0,5.0,4.0,4.375000,89.0,79,Med-Surg ICU,133
2,1494954,243334,61,199.0,66.0,5.0,4.0,4.375000,89.0,79,Med-Surg ICU,133
3,1494906,243334,66,199.0,71.0,5.0,4.0,4.375000,89.0,79,Med-Surg ICU,133
4,1494910,243334,71,199.0,76.0,5.0,4.0,4.375000,89.0,79,Med-Surg ICU,133
...,...,...,...,...,...,...,...,...,...,...,...,...
3941958,950941,3353251,12179,65.0,12184.0,5.0,NaN,17.939394,73.0,458,Cardiac ICU,1104
3941959,951041,3353251,12184,65.0,12189.0,5.0,NaN,17.939394,73.0,458,Cardiac ICU,1104
3941960,951037,3353251,12189,64.0,12194.0,5.0,NaN,17.939394,73.0,458,Cardiac ICU,1104
3941961,951045,3353251,12194,60.0,12199.0,5.0,NaN,17.939394,73.0,458,Cardiac ICU,1104


In [16]:
# add amount of pateints per hospital
bp_hospital = bp.groupby('hospitalid').agg({'stay_id': ['nunique']})
bp['pat_per_hos'] = bp["hospitalid"].map(bp_hospital[('stay_id', 'nunique')])


In [26]:
#combine all hospitals with less then 100 patients
bp['hospitalid'] = np.where(bp['pat_per_hos'] <= 100, 'under_100', bp['hospitalid'])

# bp_over_100 = bp[bp['pat_per_hos'] > 100]
# # Calculate average of columns in df_under_100
# bp_under_100 = bp_under_100.mean()
# bp_under_100 = pd.DataFrame(bp_under_100)
# # sum_bp = pd.concat([sum_bp_under_100, sum_bp_over_100])
bp

,Unnamed: 0,stay_id,cur_bp_time,cur_bp,next_bp_time,interval,drugamount,drugrate,age,hospitalid,unittype,wardid,pat_per_hos,treatment_time,avg_bp
0,1494922,243334,51,199.0,56.0,5.0,4.0,4.375000,89.0,79,Med-Surg ICU,133,126,5170.0,68.522705
1,1494950,243334,56,199.0,61.0,5.0,4.0,4.375000,89.0,79,Med-Surg ICU,133,126,5170.0,68.522705
2,1494954,243334,61,199.0,66.0,5.0,4.0,4.375000,89.0,79,Med-Surg ICU,133,126,5170.0,68.522705
3,1494906,243334,66,199.0,71.0,5.0,4.0,4.375000,89.0,79,Med-Surg ICU,133,126,5170.0,68.522705
4,1494910,243334,71,199.0,76.0,5.0,4.0,4.375000,89.0,79,Med-Surg ICU,133,126,5170.0,68.522705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941958,950941,3353251,12179,65.0,12184.0,5.0,NaN,17.939394,73.0,under_100,Cardiac ICU,1104,37,12175.0,77.432744
3941959,951041,3353251,12184,65.0,12189.0,5.0,NaN,17.939394,73.0,under_100,Cardiac ICU,1104,37,12175.0,77.432744
3941960,951037,3353251,12189,64.0,12194.0,5.0,NaN,17.939394,73.0,under_100,Cardiac ICU,1104,37,12175.0,77.432744
3941961,951045,3353251,12194,60.0,12199.0,5.0,NaN,17.939394,73.0,under_100,Cardiac ICU,1104,37,12175.0,77.432744


In [18]:


interval_total = bp.groupby('stay_id')['interval'].sum()
bp['treatment_time'] = bp["stay_id"].map(interval_total)
avg_bp = bp.groupby('stay_id')['cur_bp'].mean()
bp['avg_bp'] = bp["stay_id"].map(avg_bp)
bp
sum_bp = bp.groupby(
    'hospitalid').agg(
        {'unittype': ['nunique'],
        'stay_id': ['nunique'],
        'age': ['mean'],
        'interval': ['mean'],
        'treatment_time': ['mean'],
        'avg_bp': ['var', 'mean'],
        'drugrate': ['var', 'mean']}).sort_values(by=('stay_id', 'nunique'))
sum_bp




unittype stay_id        age  interval treatment_time     avg_bp  \
            nunique nunique       mean      mean           mean        var   
hospitalid                                                                   
243               4     114  60.426641  6.225179   12098.834102  99.382606   
165               2     120  57.411615  5.260023   10004.765900  61.445245   
199               2     120  58.290614  5.334470    8400.689451  73.172058   
79                1     126  58.570590  5.578284   10110.761087  73.665559   
252               4     133  63.384266  5.136824   24273.969911  58.499992   
188               4     138  63.577448  5.369926    8304.194747  79.291266   
154               1     144  65.852398  5.299490   10949.492800  73.459881   
157               1     155  65.364992  5.232844    9732.698057  79.325492   
148               2     156  69.738650  5.269701   14609.881372  64.993329   
176               2     199  62.334855  5.590198   16944.952920  72.612244   
167               3     238  58.145057  5.481799   21732.660922  81.826991   
420               4     342  61.751584  5.974187   16991.313178  69.543401   
under_100         8    1454  62.831447  5.503128   14163.725703  81.478489   

                          drugrate             
                 mean          var       mean  
hospitalid                                     
243         75.069264   194.878773  20.081561  
165         76.422228   100.641386  16.162838  
199         76.736051   514.423020  33.896139  
79          75.924464    78.891924   7.814060  
252         77.024183  1093.623216  33.073342  
188         72.818618    43.736542  11.514647  
154         79.198457   216.004831  20.240294  
157         74.540177   386.840516  22.101191  
148         78.890141   158.629348  14.198516  
176         78.370328   230.146376  15.910417  
167         74.960697   121.500076  14.335860  
420         77.386303     0.036211   0.183176  
under_100   76.203533  5097.660309  30.899013

In [19]:
sum_bp.drop(sum_bp.index[-2], inplace=True)


In [20]:
sum_bp

unittype stay_id        age  interval treatment_time     avg_bp  \
            nunique nunique       mean      mean           mean        var   
hospitalid                                                                   
243               4     114  60.426641  6.225179   12098.834102  99.382606   
165               2     120  57.411615  5.260023   10004.765900  61.445245   
199               2     120  58.290614  5.334470    8400.689451  73.172058   
79                1     126  58.570590  5.578284   10110.761087  73.665559   
252               4     133  63.384266  5.136824   24273.969911  58.499992   
188               4     138  63.577448  5.369926    8304.194747  79.291266   
154               1     144  65.852398  5.299490   10949.492800  73.459881   
157               1     155  65.364992  5.232844    9732.698057  79.325492   
148               2     156  69.738650  5.269701   14609.881372  64.993329   
176               2     199  62.334855  5.590198   16944.952920  72.612244   
167               3     238  58.145057  5.481799   21732.660922  81.826991   
under_100         8    1454  62.831447  5.503128   14163.725703  81.478489   

                          drugrate             
                 mean          var       mean  
hospitalid                                     
243         75.069264   194.878773  20.081561  
165         76.422228   100.641386  16.162838  
199         76.736051   514.423020  33.896139  
79          75.924464    78.891924   7.814060  
252         77.024183  1093.623216  33.073342  
188         72.818618    43.736542  11.514647  
154         79.198457   216.004831  20.240294  
157         74.540177   386.840516  22.101191  
148         78.890141   158.629348  14.198516  
176         78.370328   230.146376  15.910417  
167         74.960697   121.500076  14.335860  
under_100   76.203533  5097.660309  30.899013

In [32]:
bp.head()

,Unnamed: 0,stay_id,cur_bp_time,cur_bp,next_bp_time,interval,drugamount,drugrate,age,hospitalid,unittype,wardid,pat_per_hos,treatment_time,avg_bp
0,1494922,243334,51,199.0,56.0,5.0,4.0,4.375,89.0,79,Med-Surg ICU,133,126,5170.0,68.522705
1,1494950,243334,56,199.0,61.0,5.0,4.0,4.375,89.0,79,Med-Surg ICU,133,126,5170.0,68.522705
2,1494954,243334,61,199.0,66.0,5.0,4.0,4.375,89.0,79,Med-Surg ICU,133,126,5170.0,68.522705
3,1494906,243334,66,199.0,71.0,5.0,4.0,4.375,89.0,79,Med-Surg ICU,133,126,5170.0,68.522705
4,1494910,243334,71,199.0,76.0,5.0,4.0,4.375,89.0,79,Med-Surg ICU,133,126,5170.0,68.522705


In [45]:
# TODO try and evaluate bp to find "jumps"
by_pat = bp.groupby("stay_id").agg({
    'cur_bp': ['min', 'max', 'mean', 'var', 'std'],})
by_pat["max-min"] = by_pat['cur_bp']['max'] - by_pat['cur_bp']['min']
by_pat

cur_bp                                          max-min
           min    max       mean         var        std        
stay_id                                                        
243334    47.0  199.0  68.522705  222.594020  14.919585   152.0
243643    52.0  120.0  78.521739   88.813846   9.424110    68.0
244256    13.0  126.0  99.175258  335.730784  18.322958   113.0
244763    61.0  199.0  92.856250  236.448787  15.376891   138.0
245240    37.0   99.0  54.403433   87.224471   9.339404    62.0
...        ...    ...        ...         ...        ...     ...
3352103   37.0  199.0  78.962038  158.158557  12.576111   162.0
3352634   66.0  199.0  87.418182  341.505819  18.479876   133.0
3352721   50.0  199.0  73.381576   85.158277   9.228124   149.0
3353144    0.0  199.0  90.348279  225.662394  15.022064   199.0
3353251    0.0  199.0  77.432744   69.362450   8.328412   199.0

[3439 rows x 6 columns]

In [28]:
# bp_big will consist of patients from hospitalid with more then 400 differnt patientsunitstayid
bp_big = bp[bp["hospitalid"].isin(drug.groupby("hospitalid")["patientunitstayid"].nunique().sort_values()[drug.groupby("hospitalid")["patientunitstayid"].nunique().sort_values() > 400].index)]
bp_big

,Unnamed: 0,stay_id,cur_bp_time,cur_bp,next_bp_time,interval,drugamount,drugrate,age,hospitalid,unittype,wardid,pat_per_hos,treatment_time,avg_bp


In [22]:
# num of patients in bp_big
bp_big["stay_id"].nunique()

0

In [ ]:
#TODO plot patients by age
import plotly.graph_objects as go
import numpy as np
fig = go.Figure()
fig.add_trace(go.Histogram(x= bp_big['age'], nbinsx=10))
fig.update_layout(title='Distribution of Ages',
                  xaxis_title='Age',
                  yaxis_title='dist.')
fig.show()

In [24]:
# give me a list of 5 random patients with more than 0 occurences from bp_big names pats
pats = bp_big["stay_id"].value_counts()[bp_big["stay_id"].value_counts() > 0].sample(5)

ValueError: a must be greater than 0 unless no samples are taken

In [29]:
# create a plot of bp over time for each patient using plotly subplot and add to each patient plot the norepinephrine infusion from drug table as scatter plot
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np
for i in range(10):
    # assign random patient to pat
    pat = bp_big["stay_id"].value_counts()[bp_big["stay_id"].value_counts() > 0].sample(5).index[0]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=bp_big[bp_big["stay_id"] == pat]["cur_bp_time"], y=bp_big[bp_big["stay_id"] == pat]["cur_bp"], name="MAP"))
    # scatter plot of norepinephrine infusion, not line plot
    # fig.add_trace(go.Scatter(x=drug[drug["patientunitstayid"] == pat]["infusionoffset"], y=drug[drug["patientunitstayid"] == pat]["drugrate"], name="drug", mode="markers", marker=dict(color="red")), row=i+1, col=1)
    x = drug[drug["patientunitstayid"] == pat]["infusionoffset"]
    fig.add_trace(go.Scatter(x=x, y=65 * np.ones(x.size), name="NOR", mode="markers", marker=dict(color=bp_big[bp_big["stay_id"] == pat]["drugamount"]))) # TODO added drug amount as color
    fig.update_layout(title_text="bp over time for 5 patients")
    fig.write_image(f"bp{i}.png")



ValueError: a must be greater than 0 unless no samples are taken

In [ ]:
# create histograms of bp for each patient
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
fig = make_subplots(rows=5, cols=1, subplot_titles=[str(pat) for pat in pats.index])
for i, pat in enumerate(pats.index):
    fig.add_trace(go.Histogram(x=bp_big[bp_big["stay_id"] == pat]["cur_bp"], name="bp"), row=i+1, col=1)
fig.update_layout(height=1000, width=1000, title_text="bp over time for 5 patients")
fig.write_html("bp_hist.html")
fig.show()

In [ ]:
# if bp is a guassian process, with some kernel on the time and druginfusionoffset, we can use the kernel to predict the next bp
# create a learner
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RationalQuadratic, ExpSineSquared
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import PairwiseKernel
from sklearn.gaussian_process.kernels import StationaryKernelMixin, NormalizedKernelMixin, Kernel
from sklearn.gaussian_process.kernels import Hyperparameter
# learn a model for each patient
models = [GaussianProcessRegressor(kernel=C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2)), alpha=0.1, n_restarts_optimizer=9, normalize_y=True) for pat in pats.index]
# make plot of predictions for each patient
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
fig = make_subplots(rows=5, cols=1, subplot_titles=[str(pat) for pat in pats.index])
for i, pat in enumerate(pats.index):
    # get the data for the patient
    x = pd.DataFrame({'time': bp_big[bp_big["stay_id"] == pat]["cur_bp_time"].to_numpy(), 'drug': bp_big[bp_big["stay_id"] == pat]["cur_bp_time"].to_numpy()})
    # For each bp, in a new column, put the drugAmount of the drug that was given at 5 minutes before the bp, and 0 otherwise
    x['drug'] = x['drug'].apply(lambda t: drug[(drug["patientunitstayid"] == pat) & (drug["infusionoffset"] <= t) & (drug["infusionoffset"] >= t - 5)]["drugamount"].to_numpy()[0] if drug[(drug["patientunitstayid"] == pat) & (drug["infusionoffset"] <= t) & (drug["infusionoffset"] >= t - 5)]["drugamount"].to_numpy().size > 0 else 0)
    y = bp_big[bp_big["stay_id"] == pat]["cur_bp"].to_numpy()
    #clean the data from Nan, and inf
    x = x.replace([np.inf, -np.inf], np.nan).dropna()
    y = y[x.index]

    # find first and last drug infusion
    time_of_first_drug = drug[drug["patientunitstayid"] == pat]["infusionoffset"].min()
    time_of_last_drug = drug[drug["patientunitstayid"] == pat]["infusionoffset"].max()
    # remove data that is 30 minutes before the first drug infusion, and 30 minutes after the last drug infusion
    k = (x['time'] >= time_of_first_drug - 30) & (x['time'] <= time_of_last_drug + 30)
    x = x[k]
    y = y[k]



    # split to train and test, when the test is the 10% of the latest data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=False)
    # plot the data
    fig.add_trace(go.Scatter(x=x_train['time'], y=y_train, name="bp"), row=i+1, col=1)

    # learn the model
    models[i].fit(x_train, y_train)
    # predict the test data
    y_pred = models[i].predict(x_test)
    # plot the test data predictions
    fig.add_trace(go.Scatter(x=x_test['time'], y=y_pred, name="bp_pred"), row=i+1, col=1)
    # plot the actual test data
    fig.add_trace(go.Scatter(x=x_test['time'], y=y_test, name="bp_test"), row=i+1, col=1)
    #plot the predictions for the train data as well in a different color
    y_train_pred = models[i].predict(x_train)
    fig.add_trace(go.Scatter(x=x_train['time'], y=y_train_pred, name="bp_pred_train", line=dict(color="red")), row=i+1, col=1)

    # add mse and corr for train and test to the plot
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max(), text="mse_train: " + str(mean_squared_error(y_train, y_train_pred)), showarrow=False, row=i+1, col=1)
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max() - 5, text="corr_train: " + str(np.corrcoef(y_train, y_train_pred)[0, 1]), showarrow=False, row=i+1, col=1)
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max() - 10, text="mse_test: " + str(mean_squared_error(y_test, y_pred)), showarrow=False, row=i+1, col=1)
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max() - 15, text="corr_test: " + str(np.corrcoef(y_test, y_pred)[0, 1]), showarrow=False, row=i+1, col=1)
    # add drug markers
    x = drug[drug["patientunitstayid"] == pat]["infusionoffset"]
    fig.add_trace(go.Scatter(x=x, y=65 * np.ones(x.size), name="drug", mode="markers", marker=dict(color="red")), row=i+1, col=1)

    # add predictions for learner fitted only on time and not on drug
    models[i].kernel = RBF(10, (1e-2, 1e2))
    models[i].fit(x_train[['time']], y_train)
    y_pred = models[i].predict(x_test[['time']])
    fig.add_trace(go.Scatter(x=x_test['time'], y=y_pred, name="bp_pred_time", line=dict(color="green")), row=i+1, col=1)
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max() - 20, text="mse_test_time: " + str(mean_squared_error(y_test, y_pred)), showarrow=False, row=i+1, col=1)
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max() - 25, text="corr_test_time: " + str(np.corrcoef(y_test, y_pred)[0, 1]), showarrow=False, row=i+1, col=1)
    # add corr and mse for time_learned model for train data
    y_train_pred = models[i].predict(x_train[['time']])
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max() - 30, text="mse_train_time: " + str(mean_squared_error(y_train, y_train_pred)), showarrow=False, row=i+1, col=1)
    fig.add_annotation(x=x_train['time'].max(), y=y_train.max() - 35, text="corr_train_time: " + str(np.corrcoef(y_train, y_train_pred)[0, 1]), showarrow=False, row=i+1, col=1)
    #add predictions for train data
    fig.add_trace(go.Scatter(x=x_train['time'], y=y_train_pred, name="bp_pred_train_time", line=dict(color="green")), row=i+1, col=1)


fig.update_layout(height=5000, width=1000, title_text="bp over time for 5 patients")
fig.write_html("bp_over_time.html")
fig.show()

In [ ]:
# train the model
models[i].fit(x, y)
# predict the next 1000 values
x_pred = pd.DataFrame({'time': np.linspace(x['time'].min(), x['time'].max() + 1000, 1000), 'drug': np.linspace(x['time'].min(), x['time'].max() + 1000, 1000)})
x_pred['drug'] = x_pred['drug'].apply(lambda t: drug[(drug["patientunitstayid"] == pat) & (drug["infusionoffset"] <= t) & (drug["infusionoffset"] >= t - 5)]["drugamount"].to_numpy()[0] if drug[(drug["patientunitstayid"] == pat) & (drug["infusionoffset"] <= t) & (drug["infusionoffset"] >= t - 5)]["drugamount"].to_numpy().size > 0 else 0)
y_pred, y_std = models[i].predict(x_pred, return_std=True)
fig.add_trace(go.Scatter(x=x_pred["time"], y=y_pred, name="pred"), row=i+1, col=1)
fig.add_trace(go.Scatter(x=x_pred["time"], y=y_pred + y_std, name="pred + std"), row=i+1, col=1)
fig.add_trace(go.Scatter(x=x_pred["time"], y=y_pred - y_std, name="pred - std"), row=i+1, col=1)
fig.add_trace(go.Scatter(x=x["time"], y=y, name="bp"), row=i+1, col=1)
# calculate correlation between predicted and actual
corr = np.corrcoef(y_pred, y)
fig.update_layout(title_text=f"bp over time for 5 patients, corr={corr}")

ValueError: Expected 2D array, got 1D array instead:
array=[465 485 500 560 590 605 620 680 725].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.